In [1]:
# Standard Python modules
import os, sys
import yaml
import numpy as np
import pandas as pd
import xarray as xr
import textwrap
import datetime as dt

# import geopandas as gpd

# plot styles/formatting
import seaborn as sns
import cmocean.cm as cmo
import cmocean

# matplotlib
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from mpl_toolkits.axes_grid1 import AxesGrid
import matplotlib.ticker as mticker
import matplotlib.patches as mpatches
import matplotlib.colors as mcolors
from mpl_toolkits.axes_grid1.inset_locator import mark_inset

# extras
%matplotlib inline
import metpy.calc as mpcalc
from metpy.units import units

# import personal modules
# Path to modules
sys.path.append('../modules')
# Import my modules
from ar_funcs import AR_rank

In [2]:
# Set up paths

path_to_data = '/home/dnash/SEAK_clim_data/'      # project data -- read only
path_to_out  = '../out/'       # output files (numerical results, intermediate datafiles) -- read & write
path_to_figs = '../figs/'      # figures


In [3]:
## Processing SFO
SFO = (37.5, -122.5)
AR1 = ('2022-12-26 23', '2022-12-27 23')
AR2 = ('2022-12-29 07', '2022-12-31 21')
AR3 = ('2023-01-04 10', '2023-01-05 18')
AR4 = ('2023-01-07 10', '2023-01-08 12')
AR5 = ('2023-01-09 03', '2023-01-09 18')
AR6 = ('2023-01-10 07', '2023-01-10 17')
AR7 = ('2023-01-11 11', '2023-01-13 23')
AR8 = ('2023-01-14 06', '2023-01-14 18')

AR_lst = [AR1, AR2, AR3, AR4, AR5, AR6, AR7, AR8]

## create a ds for each AR
sfo_lst = []
for i, AR in enumerate(AR_lst):
    # create date range from AR start date and end date
    # print('AR', i+1, AR[0], AR[1])
    print('AR', i+1)
    dates = pd.date_range(start=AR[0], end=AR[1], freq='1D')
    # put into pandas df
    d ={"date": dates}
    df = pd.DataFrame(data=d)
    df['day']= df['date'].dt.day.map("{:02}".format)
    df['month']= df['date'].dt.month.map("{:02}".format)
    df['year']= df['date'].dt.year
    
    # create list of daily ERA5 files for each AR
    filenames = []
    for j, row in df.iterrows():
        filenames.append('/data/downloaded/Reanalysis/ERA5/IVT/{0}/ERA5_IVT_{0}{1}{2}.nc'.format(row['year'], row['month'], row['day']))
        
    # open all files within the AR period
    era = xr.open_mfdataset(filenames, combine='by_coords')
    era = era.sel(time=slice(AR[0], AR[1]))
    sfo = era.sel(lat=SFO[0], lon=SFO[1], method='nearest')
    
    df = sfo.to_dataframe() # IVT magnitude
    
    ## calculate IVT direction
    uvec = units.Quantity(df['uIVT'].values, "m/s")
    vvec = units.Quantity(df['vIVT'].values, "m/s")
    ivtdir = mpcalc.wind_direction(uvec, vvec)
    df['ivtdir'] = ivtdir
    ivtdir_mean = df['ivtdir'].mean()
    print('IVT Direction', ivtdir_mean)
    # print(df.IVT.values)
    ## calculate AR rank
    max_IVT, ar_rank = AR_rank(df)
    ## select only the times where IVT > 250
    idx = (df.IVT >= 250)
    df = df.loc[idx]
    start_time = df.index[0]
    end_time = df.index[-1]
    duration = len(df)
    
    print('Start time:', start_time, 'End time:', end_time)
    print('Max IVT', max_IVT)
    print('Duration', duration)
    print('AR scale', ar_rank)
    
    print(' ')
    sfo_lst.append(df)

AR 1
IVT Direction 250.96565023154142
Start time: 2022-12-26 23:00:00 End time: 2022-12-27 23:00:00
Max IVT 1095.8350003803587
Duration 25
AR scale 4
 
AR 2
IVT Direction 265.6019723742144
Start time: 2022-12-29 07:00:00 End time: 2022-12-31 21:00:00
Max IVT 606.1234757333824
Duration 63
AR scale 3
 
AR 3
IVT Direction 194.7967824835239
Start time: 2023-01-04 10:00:00 End time: 2023-01-05 18:00:00
Max IVT 832.091296832289
Duration 33
AR scale 3
 
AR 4
IVT Direction 201.50953525195854
Start time: 2023-01-07 10:00:00 End time: 2023-01-08 12:00:00
Max IVT 664.6505053552835
Duration 27
AR scale 2
 
AR 5
IVT Direction 214.5520734905829
Start time: 2023-01-09 03:00:00 End time: 2023-01-09 18:00:00
Max IVT 817.5354912696415
Duration 16
AR scale 2
 
AR 6
IVT Direction 215.04321151733427
Start time: 2023-01-10 07:00:00 End time: 2023-01-10 17:00:00
Max IVT 618.886082371259
Duration 11
AR scale 1
 
AR 7
IVT Direction 198.20592841414944
Start time: 2023-01-11 11:00:00 End time: 2023-01-13 23:00:0

In [4]:
# processing LAX
LAX = (33.5, -118.0)
AR1 = ('2022-12-27 19:00', '2022-12-28 10:00')
AR2 = ('2022-12-29 16:00', '2022-12-30 20:00')
AR3 = ('2022-12-31 20:00', '2023-01-01 10:00')
AR4 = ('2023-01-09 10:00', '2023-01-11 01:00')
AR5 = ('2023-01-14 15:00', '2023-01-16 15:00')

AR_lst2 = [AR1, AR2, AR3, AR4, AR5]

## create a ds for each AR
lax_lst = []
for i, AR in enumerate(AR_lst2):
    # create date range from AR start date and end date
    # print('AR', i+1, AR[0], AR[1])
    print('AR', i+1)
    dates = pd.date_range(start=AR[0], end=AR[1], freq='1H')
    # put into pandas df
    d ={"date": dates}
    df = pd.DataFrame(data=d)
    df['day']= df['date'].dt.day.map("{:02}".format)
    df['month']= df['date'].dt.month.map("{:02}".format)
    df['year']= df['date'].dt.year
    
    # create list of daily ERA5 files for each AR
    filenames = []
    for j, row in df.iterrows():
        filenames.append('/data/downloaded/Reanalysis/ERA5/IVT/{0}/ERA5_IVT_{0}{1}{2}.nc'.format(row['year'], row['month'], row['day']))
        
    # open all files within the AR period
    era = xr.open_mfdataset(filenames, combine='by_coords')
    era = era.sel(time=slice(AR[0], AR[1]))
    sfo = era.sel(lat=LAX[0], lon=LAX[1], method='nearest')
    
    df = sfo.to_dataframe() # IVT magnitude
    
    ## calculate IVT direction
    uvec = units.Quantity(df['uIVT'].values, "m/s")
    vvec = units.Quantity(df['vIVT'].values, "m/s")
    ivtdir = mpcalc.wind_direction(uvec, vvec)
    df['ivtdir'] = ivtdir
    ivtdir_mean = df['ivtdir'].mean()
    print('IVT Direction', ivtdir_mean)
    
    ## calculate AR rank
    max_IVT, ar_rank = AR_rank(df)
    # ## select only the times where IVT > 250
    # idx = (df.IVT >= 250)
    # df = df.loc[idx]
    start_time = df.index[0]
    end_time = df.index[-1]
    duration = len(df)
    
    print('Start time:', start_time, 'End time:', end_time)
    print('Max IVT', max_IVT)
    print('Duration', duration)
    print('AR scale', ar_rank)
    
    print(' ')
    lax_lst.append(df)

AR 1
IVT Direction 265.742238516565
Start time: 2022-12-27 19:00:00 End time: 2022-12-28 10:00:00
Max IVT 584.6338022237192
Duration 16
AR scale 1
 
AR 2
IVT Direction 312.57838500685006
Start time: 2022-12-29 16:00:00 End time: 2022-12-30 20:00:00
Max IVT 383.23573777342756
Duration 29
AR scale 1
 
AR 3
IVT Direction 229.29263560810392
Start time: 2022-12-31 20:00:00 End time: 2023-01-01 10:00:00
Max IVT 515.2534163479157
Duration 15
AR scale 1
 
AR 4
IVT Direction 231.521533404828
Start time: 2023-01-09 10:00:00 End time: 2023-01-11 01:00:00
Max IVT 600.2286230136251
Duration 40
AR scale 2
 
AR 5
IVT Direction 257.16658885758164
Start time: 2023-01-14 15:00:00 End time: 2023-01-16 15:00:00
Max IVT 551.8647430864236
Duration 49
AR scale 3
 


In [5]:
lax_lst[-2]

,IVT,lat,lon,uIVT,vIVT,IWV,ivtdir
time,,,,,,,
2023-01-09 10:00:00,260.430360,33.5,-118.0,222.611222,135.159966,25.464596,238.735675
2023-01-09 11:00:00,258.770790,33.5,-118.0,220.688043,135.126273,25.278924,238.521056
2023-01-09 12:00:00,283.514284,33.5,-118.0,243.005864,146.042799,26.120041,238.994778
2023-01-09 13:00:00,317.206066,33.5,-118.0,265.681486,173.300422,27.836949,236.884136
2023-01-09 14:00:00,347.939006,33.5,-118.0,282.229770,203.489334,29.331274,234.208173
2023-01-09 15:00:00,366.342338,33.5,-118.0,289.430510,224.581141,29.755188,232.190630
2023-01-09 16:00:00,384.547174,33.5,-118.0,302.848038,236.980158,29.946452,231.956636
2023-01-09 17:00:00,404.173885,33.5,-118.0,315.773589,252.276772,29.830128,231.378061
2023-01-09 18:00:00,401.811819,33.5,-118.0,305.844619,260.598939,29.075136,229.566905
